In [ ]:
# importing libraries
import selenium
import time
import datetime
import os
import os.path
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import TimeoutException
import json
import codecs

def ga_scraper(login, password, downloadPath,
              startDate, endDate, lang, 
               startParameter, iteration):
""""   
Function ga_scraper goes to the GA home page, logins to a GA account, surfs to the User Explorer page,
sets the Date Range and exports data for every User ID in the Date Range as a json file as <UserID.json> 
in the in the specified folder.
Function ga_scraper also outputs information about its performance in the in the specified folder.
(See example of the function call in the 3d cell below).
""""
"""" 
Arguments:
login - login to a GA account (type string).
password - password to GA account (type string).
downloadPath - path to the folder to export data for every User ID.
startDate - start date of the Date Range (type string).
endDate - end date of the Date Range (type string).
lang - language of the GA account (default Russian).
startParameter - a state parameter to specify from which User ID in the table of total User IDs to start download data 
if an exception has occured (see example of the function call in the 3d cell below).
iteration - a state parameter to specify the number of iteration to download data for all User IDs 
in the table of total User IDs in the Date Range (see example of the function call in the 3d cell below).

Returns:
endParameter - a state parameter to specify the number of iteration over the table of User IDs in the Date Range
at which an exception may have araised.
static_len - number of User IDs in the Date Range.
flag - status of the function ga_scraper (success, failure, restarting).
"""" 

    # Constants
    if lang == "RU":
        audience = 'Аудитория'
        userStastics = 'Статистика по пользователям'
        word = 'из'

    # changing default download directory

    chromeOptions = webdriver.ChromeOptions()
    prefs = {"download.default_directory" : downloadPath}
    chromeOptions.add_experimental_option("prefs",prefs)
    logFileName = '{0}{2}{3}{2}{4}{2}{5}{1}'.format('log_file', '.txt', '_',
                                          startDate, endDate, iteration)
    with open(logFileName, 'w') as logFile:

        driver = webdriver.Chrome(executable_path=r"chromedriver.exe", chrome_options=chromeOptions)

        # going to the home web page
        #driver = webdriver.Chrome(executable_path=r"chromedriver.exe")
        driver.get("https://analytics.google.com/analytics/web/")

        logFile = open(logFileName, 'w')
        timePoint = datetime.datetime.now()
        logFile.writelines('{0}{1}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S"),'.\n'))
        logFile.writelines('Going to Google Analytics web page...\n')
        logFile.writelines('Parameters:\n')
        logFile.writelines('{0}{1}'.format('Start date = ', startDate))
        logFile.writelines('\n')
        logFile.writelines('{0}{1}'.format('End date = ', endDate))
        logFile.writelines('\n')


        # checking that the opened page is desired
        #assert "Analytics" in driver.title



        # log-in (passing login) and clicking next (to the password) 
        driver.find_element_by_id("identifierId").clear()
        driver.find_element_by_id("identifierId").send_keys(login)
        driver.find_element_by_id("identifierNext").click()
        logFile.writelines('Logging in...\n')


        # passing password 
        ## Unworkable code -- driver.find_element_by_id("passwrd").send_keys(password)
        ## Unworkable code -- driver.find_element_by_id("passwordNext").click()driver.find_element_by_name("password").send_keys(password)
        try:
            element = WebDriverWait(driver, 30).until(
                EC.visibility_of_element_located((By.NAME, 'password'))
            )
            logFile.writelines('Sending password...\n')
            driver.find_element_by_name("password").send_keys(password)
            element = driver.find_element_by_id('passwordNext')
            driver.execute_script("arguments[0].click();", element)
        except TimeoutException:
            logFile.writelines('The page for entering password has not opened!\n')
            timePoint = datetime.datetime.now()
            logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
            driver.quit()
            logFile.close()
            endParameter = startParameter
            static_len = None
            flag = 'Restarting'
            return endParameter, static_len, flag

        # navigating to the target web page, step 1
        try:
            element = WebDriverWait(driver, 60).until(
                EC.text_to_be_present_in_element((By.PARTIAL_LINK_TEXT, audience), audience)
            )
            logFile.writelines('Opening menu Audience (Аудитория)...\n')
            driver.find_element_by_partial_link_text(audience).click()
        except TimeoutException:
            logFile.writelines('The page with right menu Audience (Аудитория) has not opened!\n')
            timePoint = datetime.datetime.now()
            logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
            driver.quit()
            logFile.close()
            endParameter = startParameter
            static_len = None
            flag = 'Restarting'
            return endParameter, static_len, flag


        # navigating to target web page, step 2
        try:
            element = WebDriverWait(driver, 60).until(
                EC.text_to_be_present_in_element((By.PARTIAL_LINK_TEXT, userStastics), userStastics)
            )
        except TimeoutException:
            logFile.writelines('The page with right submenu User Statistics (Статистика по пользователям) has not opened!\n')
            timePoint = datetime.datetime.now()
            logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
            driver.quit()
            logFile.close()

        logFile.writelines('Opening submenu User Statistics (Статистика по пользователям)...\n')
        driver.find_element_by_partial_link_text(userStastics).click()

        # switching to the iframe by its name (galaxy) - that contains the right part of the page
        try:
            element = WebDriverWait(driver, 60).until(
                EC.visibility_of_element_located((By.NAME, 'galaxy'))
            )
            driver.switch_to.frame(driver.find_element_by_name('galaxy'))
        except TimeoutException:
            logFile.writelines('The page with User Statistics (Статистика по пользователям) has not loaded!\n')
            driver.quit()
            timePoint = datetime.datetime.now()
            logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
            logFile.close()
            endParameter = startParameter
            static_len = None
            flag = 'Restarting'
            return endParameter, static_len, flag



        # activating datepicker by finding it by classs name and clicking with executing script
        try:
            element = WebDriverWait(driver, 30).until(
                EC.visibility_of_element_located((By.CLASS_NAME, '_GArn'))
            )
        except TimeoutException:
            logFile.writelines('Failed to activate calendar!\n')
            timePoint = datetime.datetime.now()
            logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
            driver.quit()
            logFile.close()

        logFile.writelines('Activating calendar...\n')
        driver.find_element_by_class_name('_GArn').click()

        # selecting start and end dates
        driver.find_element_by_class_name('ID-datecontrol-primary-start').clear()
        driver.find_element_by_class_name('ID-datecontrol-primary-start').send_keys(startDate)
        driver.find_element_by_class_name('ID-datecontrol-primary-end').clear()
        driver.find_element_by_class_name('ID-datecontrol-primary-end').send_keys(endDate)
        driver.find_element_by_class_name('ID-apply').click()

        # displaying all the rows on the page
        try:
            element = WebDriverWait(driver, 30).until(
                EC.visibility_of_element_located((By.CLASS_NAME, 'ACTION-toggleRowShow'))
            )
        except TimeoutException:
            logFile.writelines('Failed to set 5000 rows at the page!\n')
            timePoint = datetime.datetime.now()
            logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
            driver.quit()
            logFile.close()

        logFile.writelines('Setting 5000 rows in the table on the page to be visible...\n')
        driver.find_element_by_class_name("ACTION-toggleRowShow").send_keys(5000)

        # checking the number of rows in the top level summary table --->
        # --> "Статистика по пользователям" is less than 5000
        time.sleep(5)
        valueLevel = 5000
        try:
            element = WebDriverWait(driver, 30).until(
                EC.visibility_of_element_located((By.CLASS_NAME, 'C_PAGINATION_ROWS_LONG'))
            )
            elem = driver.find_elements_by_class_name("C_PAGINATION_ROWS_LONG")
            checkSumElem = elem[0].text
        except TimeoutException:
            logFile.writelines('Failed to find # of rows in the summary table!\n')
            timePoint = datetime.datetime.now()
            logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
            driver.quit()
            logFile.close()
            endParameter = startParameter
            static_len = None
            flag = 'Restarting'
            return endParameter, static_len, flag
        
        if valueLevel < int(checkSumElem[checkSumElem.rindex(word)+len(word)+1 : len(checkSumElem)]):
            logFile.writelines('The number of records in the summary table is more than 5000!\n')
            logFile.writelines('Need to reassign dates to fully load records in the summary table!\n')
            logFile.writelines('The script stops here.\n')
            endParameter = None
            static_len = None
            flag = 'Failure'
            timePoint = datetime.datetime.now()
            logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
            driver.close()
            logFile.close()
            return endParameter, static_len, flag

        else:
            # downloading the top level summary table "Статистика по пользователям"
            driver.find_element_by_class_name("ID-exportControlButton").click()
            time.sleep(5)
            li = driver.find_elements_by_class_name("ACTION-export")
            li[2].click()
            time.sleep(5)
            # getting number of visitors' IDs from the summary table
            elem = driver.find_elements_by_class_name("C_USER_LIST_TEXT_DIV")
            static_len = len(elem)
            if static_len < startParameter:
                logFile.writelines('The summary table has not properly loaded\n')
                timePoint = datetime.datetime.now()
                logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
                driver.quit()
                logFile.close()
                endParameter = startParameter
                static_len = None
                flag = 'Restarting'
                return endParameter, static_len, flag
                
            else:
                # saving url of the page with the top level summary table "Статистика по пользователям"
                summaryTableURL = driver.current_url

                # renaming the downloaded file with the summary table to ClientSummary.xlsx IndexError
                clientSummaryFileName = '{2}{3}{0}{3}{1}{4}'.format(startDate, endDate, 'clientSummary', '_', '.xlsx')
                try:
                    os.rename('{0}{2}{1}'.format(downloadPath, os.listdir(downloadPath)[0],'/'),
                              '{0}{2}{1}'.format(downloadPath, clientSummaryFileName,'/'))
                except IndexError:
                    logFile.writelines('Failed to rename the downloaded file with the summary table to ClientSummary.xlsx.\n')
                    timePoint = datetime.datetime.now()
                    logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
                    driver.quit()
                    logFile.close()
                    endParameter = startParameter
                    static_len = None
                    flag = 'Restarting'
                    return endParameter, static_len, flag


                # loading the 1st column (IDs) from the summary table (ClientSummary.xlsx) dataFrame clientsID
                clientsID = pd.read_excel(io = '{0}{2}{1}'.format(downloadPath, clientSummaryFileName,'/'), sheet_name = 'Набор данных1',
                                             usecols = 0, dtype = {'Идентификатор клиента' : str})

                # creating the log file and storing # of visitors
                logFile.writelines('\n')
                logFile.writelines('# of visitors:\n')
                logFile.writelines('{0}'.format(len(clientsID)))
                logFile.writelines(' \n')
                logFile.writelines('{0}{1}'.format('# of elements in the html code, static_len, = ', static_len))
                logFile.writelines(' \n')
                logFile.writelines(' \n')

                logFile.writelines('# of step, ID:\n')

                for number in range(startParameter,static_len):

                    try:
                        element = WebDriverWait(driver, 360).until(EC.visibility_of_element_located((By.CLASS_NAME, 'C_USER_LIST_TEXT_DIV'))    )
                        elem = driver.find_elements_by_class_name("C_USER_LIST_TEXT_DIV")
                        elem[number].click()
                    except TimeoutException:
                        logFile.writelines('5000 rows in the table at the page has not been loaded!\n')
                        endParameter = number + 1
                        timePoint = datetime.datetime.now()
                        logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
                        driver.close()
                        logFile.close()
                        flag = 'In process'
                        return endParameter, static_len, flag


                    try:
                        item = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CLASS_NAME, '_GAGW'))    )
                        item = driver.find_elements_by_class_name("_GAGW")
                        item[5].click()
                    except TimeoutException:
                        logFile.writelines('{0}{1}{2}'.format('The page of the visitor with ID = ',
                                                           clientsID.iat[number,0],
                                                           ' has not opened to download the visitor data\n'))
                        endParameter = number + 1
                        timePoint = datetime.datetime.now()
                        logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
                        driver.close()
                        logFile.close()
                        flag = 'In process'
                        return endParameter, static_len, flag


                    while not os.path.exists('{0}{1}{2}'.format(downloadPath, '/','user-report-export.json')):
                        time.sleep(5)
                    if os.path.isfile('{0}{1}{2}'.format(downloadPath, '/','user-report-export.json')):
                        os.rename('{0}{1}{2}'.format(downloadPath, '/','user-report-export.json'),
                                  '{0}{2}{1}{3}'.format(downloadPath, clientsID.iat[number,0],'/','.json'))
                    else:
                        raise ValueError("%s isn't a file!" % '{0}{1}{2}'.format(downloadPath, '/','user-report-export.json'))

                    logFile.writelines('{0}{2}{1}{3}'.format(number,clientsID.iat[number,0],'. ', '.\n'))
                    time.sleep(5)
                    driver.get(summaryTableURL)
                    time.sleep(5)


                    try:
                        element = WebDriverWait(driver, 360).until(EC.visibility_of_element_located((By.NAME, 'galaxy'))    )
                        driver.switch_to.frame(driver.find_element_by_name('galaxy'))
                        endParameter = number + 1
                    except TimeoutException:
                        logFile.writelines('5000 rows in the table at the page has not been reloaded!\n')
                        timePoint = datetime.datetime.now()
                        logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
                        logFile.close()
                        driver.close()
                        endParameter = number + 1
                        flag = 'In process'
                        return endParameter, static_len, flag

                #    time.sleep(15)

        # closing the log file
        timePoint = datetime.datetime.now()
        logFile.writelines('{0}'.format(timePoint.strftime("%d-%m-%Y %H:%M:%S")))
        logFile.close()

        driver.close()    

        flag = 'Success'

        return endParameter, static_len, flag 

In [ ]:
import os
import pandas as pd
def folderCheck(path2folder, path2checkFile, path2logFolder,
                checkFile, sheetName, columnName,
               startDate, endDate):
    folderContent = os.listdir(path2folder)
""""  
Function folderCheck checks resuls of function ga_scraper and output it in the log file:
1) the number of files with User IDs' data in the downloaded folder 
(it ought to be equal to the number of User IDs in the table of total User IDs).
2) the availability of data in every file with User ID' data.
3) the readability of data in every file with User ID' data.
(See the example of the function call in the 4th cell below)
""""  
""""  
Arguments:
path2folder - path the folder that contains files with User IDs' data.
path2checkFile - path the folder that contains an xlsx file with the table of total User IDs.
path2logFolder path the folder to output the log file.
checkFile - the name of xlsx file with the table of total User IDs.
sheetName - the name of the sheet in the xlsx file with the table of total User IDs.
columnName the name of the columns that contains IDs in the table of total User IDs
startDate - start date of the Date Range (type string).
endDate - end date of the Date Range (type string).

Returns:
folderCheckFile - file with check resuls
""""  
    
    if path2folder == path2checkFile or path2checkFile == None:
        folderContent.remove(checkFile)
    else:
        pass
#    folderContent.sort
    folderCheckFile = pd.read_excel(io = path2checkFile + '/' + checkFile, 
                              sheet_name = sheetName,
                              usecols = 0, dtype = {columnName : str})
#    folderCheckFile.sort_values(by = columnName, inplace = True)
    folderCheckFile['Missing'] = ''
    counter1 = []
    for j in range(len(folderCheckFile)):
        if folderCheckFile.iat[j,0] + '.json' in folderContent:
            folderCheckFile.iat[j,1] = True
        else:
            folderCheckFile.iat[j,1] = False
            counter1.append(j)

    folderCheckFileList = folderCheckFile[columnName].values.tolist()
    counter2 = []
    for j in range(len(folderContent)):
        if folderContent[j].replace('.json', '') in folderCheckFileList:
            pass        
        else:
            counter2.append(folderContent[j])
            
    outputFileName = '{5}{0}{2}{3}{2}{4}{1}'.format('folderCheck', '.txt', '_',
                                                 startDate, endDate, path2logFolder)
    
    logFile = open(outputFileName, 'w')
    logFile.writelines('Parameters:\n')
    logFile.writelines('{0}{1}'.format('Start date = ', startDate))
    logFile.writelines('\n')
    logFile.writelines('{0}{1}'.format('End date = ', endDate))
    logFile.writelines('\n')
    if counter1 == []:
        logFile.writelines('The number and names of file in the folder are correct. \n')
    else:
        logFile.writelines('{0}{1}{2}'.format('1. ', len(counter1),
                                              ' json files are missing: \n'))
        for k in range(len(counter1)):
            logFile.writelines('{0}{1}{2}{3}{4}'.format('   ', k+1, '). ',
                                                        clientsID.iat[counter1[k],0],
                                                        '.json.\n'))

    logFile.writelines('\n')
            
    if counter2 == []:
        pass
    else:
        logFile.writelines('{0}{1}{2}'.format('2. ', len(counter2),
                                              ' json files are not in the IDs list: \n'))
        for k in range(len(counter2)):
            logFile.writelines('{0}{1}{2}{3}{4}'.format('   ', k+1, '). ',
                                                        [counter2[k]],
                                                        '.\n'))
    logFile.close
    return folderCheckFile

In [ ]:
# EXAMPLE of the ga_scraper FUNCTION CALL

import os
import datetime

login =''
password = ''
               
lang = "RU"
startDate = [''] #['dd.mm.yyyy']
endDate = [''] #['dd.mm.yyyy']
dPath = ['']

startParameter = 0 
iteration = 0
       
timePoint = datetime.datetime.now()
print('Запуск расчетов')
print (timePoint.strftime("%d-%m-%Y %H:%M:%S"))
print('\n')

result = ga_scraper(login, password, dPath, startDate, endDate, lang, startParameter, iteration)
timePoint = datetime.datetime.now()
print (timePoint.strftime("%d-%m-%Y %H:%M:%S"))
print('Результат расчета функции')
print(result)
print('\n')

if result[2] == 'Failure':
    print('Script has stopped, see its log file.')
else:
    if result[0] == result[1]:
        print('OK')
    else:
        while result[0] != result[1]:
            iteration = iteration + 1
            iterationPath = dPath + '_' + str(iteration)


            os.makedirs(iterationPath)
            startParameter = result[0]
            result = ga_scraper(login, password, iterationPath, startDate, endDate, lang, startParameter, iteration)
            timePoint = datetime.datetime.now()
            print (timePoint.strftime("%d-%m-%Y %H:%M:%S"))
            print('Результат расчета функции')
            print(result)
            print('{0}{1}'.format('Исходная папка: ', dPath))
            print('{0}{1}'.format('Папка для записи файлов: ', iterationPath))
            print ('\n')



In [ ]:
import os
fileName = ''
path2folder = path2checkFile = ''
path2logFolder = ''
startDate = '' #['dd.mm.yyyy']
endDate = '' #['dd.mm.yyyy']
sheetName = ''
columnName = ''

clientsID = pd.read_excel(io = path2folder + '/' + fileName, 
                          sheet_name = sheetName,
                          usecols = 0, dtype = {columnName : str})

result = folderCheck(path2folder, path2checkFile, path2logFolder, fileName, 
                     sheetName, columnName, startDate, endDate)